# 流程控制概述

Runnable 可以支持包括顺序、分支、条件、迭代等多种控制方式，构建较复杂的有向无环图。

与直接使用python代码控制流程不同的是，下面这些流程控制手段仍然保持以Runnable返回，以便获得LCEL的诸多额外好处，如：Runnable的统一方法、序列化能力、重试能力、回滚能力、Langsmith追踪和Langserve的API集成等。

# 流程控制相关类结构

- Runnable
    - RunnableSerializable
        - RunnableSequence（实现顺序执行，可以用|符号或RunnableSequence来构造）
        - RunnableParallel（实现并行执行，可以用Dict或RunnableParallel类来构造，别名RunnableMap）
        - RunnableEach（实现相同的链迭代多次）
        - RunnableBranch（实现流程分支，根据条件选择不同的链执行）
        - RouterRunnable（实现流程分支，根据枚举值选择不同的链执行）

# RunnableSerializable

主要增加了序列化的方法，可以将Runnable以JSON字符串的方式保存或加载：

- loads / dumps （按JSON字符串）
- load / dumpd （按Dict类型）

## 将Runnable保存为JSON

In [95]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableGenerator
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from typing import Iterator

model = ChatOpenAI()
chant = (
    ChatPromptTemplate.from_template("Give me a 3 word chant about {topic}")
    | model
    | StrOutputParser()
)

In [96]:
from langchain_core.load import loads, dumps
json = dumps(chant)
print(json)

{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "chat", "ChatPromptTemplate"], "kwargs": {"input_variables": ["topic"], "messages": [{"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "chat", "HumanMessagePromptTemplate"], "kwargs": {"prompt": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["topic"], "template": "Give me a 3 word chant about {topic}", "template_format": "f-string", "partial_variables": {}}}}}]}}, "middle": [{"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}}}], "last": {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "output_parser", "StrOutputParser"], "kwargs": {}}, "name": null}}


## 从JSON中恢复Runnable并调用

In [99]:
loads(json).invoke({"topic":"苹果"})

'"Apple, sweet success!"'

## 根据配置动态选择Runnable对象

In [231]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.runnables import ConfigurableField, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Give me a 3 word chant about {topic}")
openai = ChatOpenAI()
llm = OpenAI()
output_parser = StrOutputParser()

configurable_model = llm.configurable_alternatives(
    ConfigurableField(id="model"), 
    default_key="llm_openai", 
    chat_openai=openai
)

configurable_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | configurable_model 
    | output_parser
)

In [192]:
configurable_chain.invoke(
    "ice cream", 
    config={"model": "llm_openai"}
)

'\n\n"Sweet, cold, yum!"'

In [193]:
configurable_chain.invoke(
    "ice cream", 
    config={"model": "chat_openai"}
)

'\n\n"Scoop, swirl, savor!"'

## 查看有哪些动态选择

# RunnableSequence

RunnableSequence 是最最重要的流程控制组件。

另外，RunnableSequence 也增加了几个属性：
- first: 第一个Runnable组件
- middle: 中间的Runnable组件列表
- last: 最后一个Runnable组件

In [228]:
from langchain_core.runnables import RunnableLambda
import random

def add_one(x: int) -> int:
    return x + 1

def buggy_double(y: int) -> int:
    '''Buggy code that will fail 70% of the time'''
    if random.random() > 0.3:
        print('This code failed, and will probably be retried!')
        raise ValueError('Triggered buggy code')
    return y * 2

chain = (
    RunnableLambda(add_one) |
    RunnableLambda(buggy_double).with_retry( # Retry on failure
        stop_after_attempt = 10,
        wait_exponential_jitter = False
    )
)
chain.get_graph().print_ascii()

print("---RunnableSequence")
print(type(chain.first))
print(type(chain.middle))
print(type(chain.last))

    +---------------+    
    | add_one_input |    
    +---------------+    
            *            
            *            
            *            
  +-----------------+    
  | Lambda(add_one) |    
  +-----------------+    
            *            
            *            
            *            
+----------------------+ 
| Lambda(buggy_double) | 
+----------------------+ 
            *            
            *            
            *            
+---------------------+  
| buggy_double_output |  
+---------------------+  
---RunnableSequence
<class 'langchain_core.runnables.base.RunnableLambda'>
<class 'list'>
<class 'langchain_core.runnables.retry.RunnableRetry'>


# RunnableParallel

一种情况是从同一个输入生成多个分支链：

In [227]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

def mul_three(x: int) -> int:
    return x * 3

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
runnable_3 = RunnableLambda(mul_three)

sequence = runnable_1 | {  # this dict is coerced to a RunnableParallel
    "mul_two": runnable_2,
    "mul_three": runnable_3,
}
# Or equivalently:
# sequence = runnable_1 | RunnableParallel(
#     {"mul_two": runnable_2, "mul_three": runnable_3}
# )
# Also equivalently:
# sequence = runnable_1 | RunnableParallel(
#     mul_two=runnable_2,
#     mul_three=runnable_3,
# )

sequence.invoke(1)
sequence.get_graph().print_ascii()

await sequence.ainvoke(1)

sequence.batch([1, 2, 3])
await sequence.abatch([1, 2, 3])

                    +---------------+                     
                    | add_one_input |                     
                    +---------------+                     
                            *                             
                            *                             
                            *                             
                  +-----------------+                     
                  | Lambda(add_one) |                     
                  +-----------------+                     
                            *                             
                            *                             
                            *                             
          +----------------------------------+            
          | Parallel<mul_two,mul_three>Input |            
          +----------------------------------+            
                  ***               ***                   
               ***                     ***              

[{'mul_two': 4, 'mul_three': 6},
 {'mul_two': 6, 'mul_three': 9},
 {'mul_two': 8, 'mul_three': 12}]

# RunnableEach

如果需要让同一个链执行多次，使用 RunnableEach 非常方便。

In [224]:
from langchain_core.runnables.base import RunnableEach, chain

@chain
def myfunc(x):
    return(x * 2)

runnable_each = RunnableEach(bound = myfunc)
runnable_each.get_graph().print_ascii()

output = runnable_each.invoke(range(1, 5))
print(output)

 +--------------+  
 | myfunc_input |  
 +--------------+  
         *         
         *         
         *         
+----------------+ 
| Lambda(myfunc) | 
+----------------+ 
         *         
         *         
         *         
+---------------+  
| myfunc_output |  
+---------------+  
[2, 4, 6, 8]


# RunnableBranch

有时候希望实现类似 ifelse 的条件分支，使用 RunnableBrach 可以接受 (条件函数, runnable) 的元组对和一个默认分支。

In [225]:
from langchain_core.runnables import RunnableBranch, chain

@chain
def myfunc(x):
    return(x * 2)

branch = RunnableBranch(
    (lambda x: isinstance(x, str), lambda x: x.upper()),
    (lambda x: isinstance(x, int), myfunc),
    lambda x: "goodbye",
)
branch.get_graph().print_ascii()

print(branch.invoke("hello")) # "HELLO"
print(branch.invoke(3)) # 6
print(branch.invoke(None)) # "goodbye"

+-------------+  
| BranchInput |  
+-------------+  
        *        
        *        
        *        
   +--------+    
   | Branch |    
   +--------+    
        *        
        *        
        *        
+--------------+ 
| BranchOutput | 
+--------------+ 
HELLO
6
goodbye


# RouterRunnable

条件分支中还有一种情况更为简单，适合使用RouterRunnable，可以比RunnableBranch简洁得多。
<br>但传递参数时必须以 RouterInput 类型传入，或使用等价的字典结构。

```
class RouterInput(TypedDict):
    key: str
    input: Any
```

In [226]:
from langchain_core.runnables import RouterRunnable, chain

@chain
def lower(x):
    return(x.lower())

@chain
def upper(x):
    return(x.upper())

router = RouterRunnable({"lower": lower, "upper": upper})
router.get_graph().print_ascii()

print(router.invoke({"key": "upper", "input": "Hello"}))
print(router.invoke({"key": "lower", "input": "Hello"}))

+---------------------+  
| RouterRunnableInput |  
+---------------------+  
            *            
            *            
            *            
   +----------------+    
   | RouterRunnable |    
   +----------------+    
            *            
            *            
            *            
+----------------------+ 
| RouterRunnableOutput | 
+----------------------+ 
HELLO
hello
